# **Esercitazione 2 - Regressione Lineare**

## Boston Housing dataset

Questo dataset contiene informazioni raccolte dal U.S. Census Service riguardanti le abitazioni nell'area di Boston, Massachusetts. È stato ottenuto dall'archivio StatLib (http://lib.stat.cmu.edu/datasets/boston) ed è stato ampiamente utilizzato in letteratura per fare benchmark di algoritmi. 

Il dataset contiene informazioni su 506 case, divise in 14 variabili.

In [147]:
import numpy as np
import matplotlib.pyplot as plt

In [148]:
import pandas as pd 
from sklearn.utils import shuffle
from pandas import read_csv

from sklearn.datasets import fetch_openml
import pandas as pd

# Scarica il Boston Housing Dataset da OpenML
boston = fetch_openml(name="Boston", version=1, as_frame=True, parser="auto")


# Estrai i dati (features) e il target (valore mediano delle abitazioni)
X = boston.data
y = boston.target

X, y = shuffle(X, y, random_state=0)
print(f"Features shape: {X.shape}, targets shape:  {y.shape}")

Features shape: (506, 13), targets shape:  (506,)


## `np.c_` in NumPy

L'oggetto `np.c_` in NumPy è una **scorciatoia** per concatenare array lungo il secondo asse (cioè, le colonne).

## Utilizzo
```python
np.c_[array1, array2, ...]


In [149]:
import numpy as np

# Generate two random 2x3 matrices
matrice1 = np.random.rand(2, 3)
matrice2 = np.random.rand(2, 3)

# Concatenate the matrices along columns
risultato = np.c_[matrice1, matrice2]

print("Matrice 1:",matrice1.shape)

print("\nMatrice 2:",matrice2.shape)

print("\nMatrice concatenata:",risultato.shape)

Matrice 1: (2, 3)

Matrice 2: (2, 3)

Matrice concatenata: (2, 6)


**Divisione del dataset**

Il primo passaggio è quello di dividere i dati in train set, validation set e test set. Utilizza il 60% dei dati per il training set, il 20% per il validation e il restante 20% per il test set. Considerato che il nostro dataset possiede 506 osservazioni mi aspetto che:

- Il **training set** avrà 303 osservazioni.
- Il **validation set** avrà 101 osservazioni.
- Il **test set** avrà 101 osservazioni.

In reatà il test set avrà 102 osservazioni per via delle approssimazioni.



In [150]:
# Divisione del dataset

tasso_addestramento = 0.6  
tasso_validazione = 0.2  
tasso_test = 0.2

# svolgimento...
dati_completi = np.c_[X, y]

n_total = dati_completi.shape[0]
n_train = int(n_total * 0.6)
n_val = int(n_total * 0.2)
n_test = n_total - n_train - n_val

train_set = dati_completi[:n_train]
val_set = dati_completi[n_train:n_train + n_val]
test_set = dati_completi[n_train + n_val:]

X_train, y_train = train_set[:, :-1], train_set[:, -1]
X_val, y_val = val_set[:, :-1], val_set[:, -1]
X_test, y_test = test_set[:, :-1], test_set[:, -1]

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(303, 13) (303,)
(101, 13) (101,)
(102, 13) (102,)


### **Esercizio 1: Costruisci una Pipeline di Regressione Lineare Standardizzata**

**Step 1:** Standardizza i dataset di addestramento, validazione e test. Usa `StandardScaler` di scikit-learn.  

**Step 2:** Aggiungi una feature costante (bias) ai dati concatenando una colonna di uno ad ogni dataset.  

**Step 3:** Implementa la soluzione in forma chiusa per l'addestramento di un modello di regressione lineare. 
 
**Step 4:** Valuta il modello calcolando il Mean Absolute Error (MAE) sui dataset di addestramento, validazione e test.


### **Guida**

1. **StandardScaler**:
   - Utilizza `StandardScaler` da `sklearn.preprocessing` per standardizzare i dati.
   - Il metodo `fit_transform` calcola la media e la varianza dei dati di addestramento e li scala di conseguenza.
   - Utilizza `transform` per standardizzare i dati di validazione e test utilizzando gli stessi parametri. Utilizziamo il metodo `transform` perchè non calcola i parametri di scaling (media e std). In questo modo ci assicuriamo che i dati di training e quelli di validation e test vengano scalati in modo uguale. Se usassimo `fit_transform` avremmo degli scaling diversi.

2. **Aggiunta di una Caratteristica Costante**:
   - Utilizza `np.c_` per concatenare una colonna di uno alle matrici delle caratteristiche. Questo è importante per includere il termine di intercetta nella regressione lineare.

3. **Soluzione in Forma Chiusa per la Regressione Lineare**:
   - La soluzione in forma chiusa è:

     $$\theta = (X^T X)^{-1} X^T y$$

   - Per calcolare la trasposta di una matrice possiamo utilizzare l' attributo `.T` di cui ogni array è dotato.

   - Utilizza `np.linalg.inv` di NumPy per l'inversione della matrice e l'operatore `@` per la moltiplicazione matriciale.
  
   - Puoi utilizzare l'operatore @ per eseguire l'operazione np.dot (`A @ B` è equivalente a `np.dot(A, B)`).

4. **Mean Absolute Error (MAE)**:
   - L'MAE si calcola come:

     $$\text{MAE} = \frac{1}{n} \sum_{i=1}^n |y_i - \hat{y}_i|$$

   - Utilizza `np.mean` e `np.abs` per calcolarlo.


In [151]:
# Step 1 - Normalizzazione dei dati. Dobbiamo normalizzare le features 
# sia del training set, validation set e test set.

# Utilizziamo il metodo .fit_transform() dello scaler per normalizzare le feature di training.

# Per normalizzare le feature di validation e test utilizziamo il metodo .transform()

from sklearn.preprocessing import StandardScaler

# svolgimento...

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled.shape,X_val_scaled.shape,X_test_scaled.shape)

(303, 13) (101, 13) (102, 13)


In [152]:
# Step 2 - Aggiunta di una feature costante

# creiamo un vettore di 1 da aggiungere come feature costante. 
# ATTENZIONE: questo vettore deve avere le stesse righe del set a cui viene aggiunto. 
# Uno uguale per tutti non va bene

# svolgimento...

ones_train = np.ones((X_train_scaled.shape[0], 1))
ones_val = np.ones((X_val_scaled.shape[0], 1))
ones_test = np.ones((X_test_scaled.shape[0], 1))

X_train_final = np.c_[X_train_scaled, ones_train]
X_val_final = np.c_[X_val_scaled, ones_val]
X_test_final = np.c_[X_test_scaled, ones_test]

print(X_train_final.shape, X_val_final.shape, X_test_final.shape)
#print(X_train, X_val, X_test)

(303, 14) (101, 14) (102, 14)


In [153]:
# Step 3 - Applichiamo la formula matematica della regressione lineare

# ATTENZIONE: stiamo per effettuare operazioni tra matrici e vettori, 
# non si tratta di una semplice formula matematica, stiamo attenti a quali operatori utilizzare e quanto

# svolgimento...

X_transpose = X_train_final.T
X_transpose_X = X_transpose @ X_train_final
X_transpose_X_inv = np.linalg.inv(X_transpose_X)
beta = X_transpose_X_inv @ X_transpose @ y_train

print("Coefficienti Beta:")
print(beta)

Coefficienti Beta:
[-0.5092644895060656 1.045701672880109 0.08640981953958554
 0.4301427277594411 -1.8975044049613647 2.629644778449658
 -0.024366370917105154 -2.9544103200356466 2.928927848145119
 -2.41571655722088 -1.7414469784098787 1.1213299373153893
 -3.5654356560981757 22.16138613861386]


In [154]:
# Step 4 - Calcolo MAE

# Calcoliamo l'errore medio assoluto (MAE) per il training set, validation set e test set.
# Utlizziamo la formula specificata nella guida.

# svolgimento...

def calculate_mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

y_train_pred = X_train_final @ beta
y_val_pred = X_val_final @ beta
y_test_pred = X_test_final @ beta

mae_train = calculate_mae(y_train, y_train_pred)
mae_val = calculate_mae(y_val, y_val_pred)
mae_test = calculate_mae(y_test, y_test_pred)

print(f"MAE Training Set: {mae_train}")
print(f"MAE Validation Set: {mae_val}")
print(f"MAE Test Set: {mae_test}")

MAE Training Set: 3.2496837441036988
MAE Validation Set: 3.5977784744283947
MAE Test Set: 3.0197366706166866


### **Esercizio: Costruisci una pipeline di Regressione Lineare Standardizzata utilizzando `scikit-learn`** 

**Step 1 & 2:** Step 1 e 2 sono uguali a quanto fatto prima.

**Step 3:** Utilizza `LinearRegression()` di scikit-learn per addestrare un modello di regressione lineare.  

**Step 4:** Valuta il modello calcolando il Mean Absolute Error (MAE) sui dataset di addestramento, validazione e test, utilizzando `mean_absolute_error()` da `sklearn.metrics`.


## `LinearRegression` da Scikit-Learn

La classe `LinearRegression` in Scikit-Learn viene utilizzata per eseguire la **regressione lineare**, adattando un modello lineare al dataset.

## **Sintassi**
```python
from sklearn.linear_model import LinearRegression

model = LinearRegression()
# Dati di esempio
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.array([10, 15, 20, 25])

# Adatta il modello ai dati
model.fit(X, y)

# Predici nuovi valori
X_new = np.array([[3, 5], [5, 9]])
predictions = model.predict(X_new)


## `mean_absolute_error` da Scikit-Learn

La funzione `mean_absolute_error` calcola l'**errore assoluto medio** (MAE) tra i valori target reali e quelli predetti.

## **Sintassi**
```python
sklearn.metrics.mean_absolute_error(y_true, y_pred)


### **Guida**

1. **Istanziare e allenare un modello di regressione lineare**:
    
    - Istanziamo una classe `LinearRegression` per creare il modello.
    - Utilizziamo il metodo `.fit()` per allenare il modello con i dati di training.

2. **Effettuare predizioni con il modello**:

    - Utiliziamo il metodo `.predict()` del modello per effettuare le predizioni. Effettuiamo le predizioni per tutti i set che abbiamo (train, validation e test).

3. **Calcolo della MAE**: 

    - Calcolare MAE su tutti i set utilizzando la funzione `mean_abslute_error`


In [155]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [156]:
# Step 1 - Istanziare e allenare il modello di regressione lineare.

# svolgimento...
model = LinearRegression()
model.fit(X_train, y_train) 

LinearRegression()

In [157]:
# Step 2 - Effettuare predizioni

# svolgimento...
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

print(y_train_pred.shape, y_val_pred.shape, y_test_pred.shape)


(303,) (101,) (102,)


In [158]:
# Step 3 - Calcolo MAE

# svolgimento...
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f"MAE Training set: {mae_train:.4f}")
print(f"MAE Validation set: {mae_val:.4f}")
print(f"MAE Test set: {mae_test:.4f}")

MAE Training set: 3.2497
MAE Validation set: 3.5978
MAE Test set: 3.0197


### **Esercizio: Crea una funzione che esegua una pipeline di Regressione Lineare**

La funzione deve richiedere un parametro `hyperparams` per gestire i diversi casi. 

`hyperparams` deve essere un dizionario contenente diverse chiavi, in base al valore di queste chiavi devono essere eseguiti (oppure no) diversi pezzi di codice. 

In questo esercizio la chiave da utilizzare sarà `hyperparams['data_standardize']`. Se il valore di questa chiave sarà **True** allora eseguire la standardizzazione con `scikit-learn`, se invece è **False** non verrà eseguita alcuna standardizzazione.

**Step 1:** Controllare se eseguire o no la standardizzazione.

* **Step 1.1:** Scrivere il codice per eseguire la standardizzazione.

**Step 2:** Utilizza `np.c_` per concatenare una colonna di uno alle matrici delle caratteristiche.

**Step 3:** Applichiamo la formula matematica della regressione lineare.

**Step 4:** Calcolo MAE utilizzando la formula (NON con `scikit-learn`).

La funzione deve ritornare i valori della MAE.

Dopo aver testato i risultati con `hyperparams['data_standardize']` = **True**, provare anche i risultati ottenuti se `hyperparams['data_standardize']` = **False**.

In [159]:
def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    # Step 1 - Controllo se è richiesta la standardizzazione dei dati
    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)  # Ora X_val viene sempre trasformato!
    
    # Step 2 - Concatenare una colonna di 1 alla matrice delle features
    X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_val = np.c_[np.ones((X_val.shape[0], 1)), X_val]

    # Step 3 - Applicare la formula della regressione lineare con pseudo-inversa
    X_train_T = X_train.T
    beta = np.linalg.pinv(X_train_T @ X_train) @ X_train_T @ y_train  # Più robusto di inv()
    
    # Step 4 - Calcolare le predizioni e il MAE
    y_val_pred = X_val @ beta
    mae = np.mean(np.abs(y_val - y_val_pred))
    
    return mae 

In [160]:
hyperparams = {'data_standardize': True}

train_fraction = 0.8
validation_fraction = 0.2

num_train = int(train_fraction * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]

X_validation = X[num_train:]
y_validation = y[num_train:]

# Chiamare la funzione pipeline e stampare i risultati della MAE

# svolgimento...
hyperparams = {'data_standardize': True}
mae = pipeline(X_train, y_train, X_validation, y_validation, hyperparams)

print(f"MAE standardizzato: {mae}")

MAE standardizzato: 3.023751047336025


### **Esercizio: Implementare alla funzione `pipeline` la possibilità di usare PCA**

Modifichiamo la funzione `pipeline` in modo da gestire anche la possibilità di effettuare la PCA. Dunque aggiungiamo al dizionario `hyperparams` la chiave `use_pca`. 

Se `hyperparams['use_pca']` = **True** verrà eseguita la PCA. 

Se `hyperparams['use_pca']` = **False** non verrà eseguita la PCA.

La gestione della standardizzazione deve essere mantenuta come prima.

In [161]:
# svolgimento...
from sklearn.decomposition import PCA

def pipeline(X_train, y_train, X_val, y_val, hyperparams):

    X_train = np.array(X_train, dtype=float)
    y_train = np.array(y_train, dtype=float)
    X_val = np.array(X_val, dtype=float)
    y_val = np.array(y_val, dtype=float)

     # Step 1 - Standardizzazione (da fare PRIMA della PCA!)
    scaler = None
    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Step 2 - PCA (se richiesta)
    pca = None
    if hyperparams['use_pca']:
        pca = PCA(n_components=0.95)  # Manteniamo il 95% della varianza
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)

    # Step 3 - Concatenare una colonna di 1 alla matrice delle features
    X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_val = np.c_[np.ones((X_val.shape[0], 1)), X_val]

    # Step 4 - Applicare la formula della regressione lineare
    theta = np.linalg.pinv(X_train.T @ X_train) @ X_train.T @ y_train  # Più stabile di inv()

    # Step 5 - Calcolare le predizioni e il MAE
    y_pred = X_val @ theta
    mae = np.mean(np.abs(y_pred - y_val))

    return mae, theta, scaler, pca

In [162]:
hyperparams_pca = {'data_standardize': True, 'use_pca': True}
hyperparams_no_pca = {'data_standardize': True, 'use_pca': False}

train_fraction = 0.8
num_train = int(train_fraction * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]
X_validation = X[num_train:]
y_validation = y[num_train:]

mae_pca = pipeline(X_train, y_train, X_validation, y_validation, hyperparams_pca)[0]
mae_no_pca = pipeline(X_train, y_train, X_validation, y_validation, hyperparams_no_pca)[0]

print(f"MAE con PCA: {mae_pca:.3f}")
print(f"MAE senza PCA: {mae_no_pca:.3f}")

MAE con PCA: 3.157
MAE senza PCA: 3.024
